<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Подготовить-процессы-обработки-данных" data-toc-modified-id="Подготовить-процессы-обработки-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовить процессы обработки данных</a></span></li><li><span><a href="#Создать-пакет-данных-и-итератор" data-toc-modified-id="Создать-пакет-данных-и-итератор-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Создать пакет данных и итератор</a></span></li><li><span><a href="#Определите-модель" data-toc-modified-id="Определите-модель-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Определите модель</a></span></li><li><span><a href="#Инициировать-экземпляр" data-toc-modified-id="Инициировать-экземпляр-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Инициировать экземпляр</a></span></li><li><span><a href="#Определите-функции-для-обучения-модели-и-оценки-результатов." data-toc-modified-id="Определите-функции-для-обучения-модели-и-оценки-результатов.-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Определите функции для обучения модели и оценки результатов.</a></span></li><li><span><a href="#Разделите-набор-данных-и-запустите-модель" data-toc-modified-id="Разделите-набор-данных-и-запустите-модель-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Разделите набор данных и запустите модель</a></span></li><li><span><a href="#Оцените-модель-с-помощью-тестового-набора-данных" data-toc-modified-id="Оцените-модель-с-помощью-тестового-набора-данных-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Оцените модель с помощью тестового набора данных</a></span></li><li><span><a href="#Тест-на-случайной-новости" data-toc-modified-id="Тест-на-случайной-новости-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Тест на случайной новости</a></span></li></ul></div>

In [55]:
%matplotlib inline


Классификация текста с помощью библиотеки torchtext
==================================

В этом руководстве мы покажем, как использовать библиотеку torchtext для создания набора данных для анализа классификации текста. Пользователи будут иметь возможность


    - Доступ к необработанным данным в качестве итератора
    - Создание конвейера обработки данных для преобразования необработанных текстовых строк в torch.Tensor, который можно использовать для обучения модели.
    - Перемешивайте и повторяйте данные с помощью `torch.utils.data.DataLoader
    
<https://pytorch.org/docs/stable/data.html?highlight=dataloader#torch.utils.data.DataLoader>`__


Загрузка данных
-----------------------------------

In [56]:
import torch
import pandas as pd

In [57]:
# Список категорий в тестовых данных
# pd.read_pickle('../data/input/test_data_4300_rows.pkl')['category_id'].drop_duplicates().values

In [58]:
df = pd.read_pickle('../data/input/test_data_all_rows.pkl')[['category_id', 'item_name']]
df[df.item_name == 'Молоко ДЕРЕВЕНСКОЕ МОЛОЧКО пастер 2.5% п/пак 900мл']

,category_id,item_name
43146,78,Молоко ДЕРЕВЕНСКОЕ МОЛОЧКО пастер 2.5% п/пак 9...


In [59]:
def open_data(path, cat_dict = {}):
    df = pd.read_pickle(path)[[
                            'category_id', 'item_name']]
#     df = df[~df.category_id.isin([0, 133, 3, 177, 43, 18, 120, 45, 2, 49, 66, 30, 118, 7])]
    if cat_dict == {}:
        cat_num = 0
        for category_id in  df.category_id.drop_duplicates():
            cat_dict[category_id] = cat_num
            df.loc[df.category_id == category_id, 'category_id_new'] = cat_num
            cat_num += 1
        return df, cat_dict
    else:
        df['category_id_new'] = df.category_id.map(cat_dict)
        return df
    

In [60]:
train_df, cat_dict = open_data('../data/input/test_data_all_rows.pkl')
train_data = train_df[['category_id_new', 'item_name']].to_numpy()
print(cat_dict)
print(train_data.shape)
train_data

{78: 0, 71: 1, 70: 2, 84: 3, 69: 4, 68: 5, 40: 6, 203: 7, 79: 8, 7: 9, 117: 10, 80: 11, 83: 12, 53: 13, 73: 14, 81: 15, 0: 16, 12: 17, 85: 18, 77: 19, 145: 20, 2: 21, 204: 22, 139: 23, 38: 24, 74: 25, 130: 26, 75: 27, 49: 28, 45: 29, 133: 30, 76: 31, 82: 32, 19: 33, 43: 34, 51: 35, 61: 36, 177: 37, 118: 38, 92: 39, 36: 40, 30: 41, 167: 42, 66: 43, 52: 44, 107: 45, 37: 46, 3: 47, 72: 48, 62: 49, 50: 50, 120: 51, 42: 52, 150: 53, 57: 54, 6: 55, 140: 56, 101: 57, 163: 58, 20: 59, 103: 60, 4: 61, 31: 62, 67: 63, 27: 64, 29: 65, 114: 66, 102: 67, 115: 68, 26: 69, 35: 70, 39: 71, 13: 72, 9: 73, 128: 74, 60: 75, 41: 76, 138: 77, 11: 78, 100: 79, 24: 80, 96: 81, 109: 82, 106: 83, 56: 84, 143: 85, 105: 86, 90: 87, 1: 88, 108: 89, 164: 90, 55: 91, 111: 92, 58: 93, 54: 94, 97: 95, 46: 96, 121: 97}
(48267, 2)


array([[0.0, 'Молоко 3,2%,шт'],
       [1.0, 'Компот из изюма, 114 ккал'],
       [1.0, 'Макаронные изделия отварные (масло сливочное), 240 ккал'],
       ...,
       [16.0, 'Пиво светлое "Халзан" 4,5 % об, пл/б. 1,5 л(шт)'],
       [52.0, 'Экспресс педикюр'],
       [3.0, 'Конфеты Харитоша 1кг мол. ваф Яшкино']], dtype=object)

In [61]:
test_df = open_data('../data/input/test_data_6350_rows.pkl',cat_dict = cat_dict)
test_data = test_df[['category_id_new', 'item_name']].to_numpy()
test_data

array([[0, 'Молоко ДЕРЕВЕНСКОЕ МОЛОЧКО пастер 2.5% п/пак 900мл'],
       [11, 'Мандарины с листиком 1кг'],
       [3, 'Хлеб/бул изделие 200г отруби  ХБЗ №2'],
       ...,
       [7, 'Пакет'],
       [19, '230Г СОУС КАЛЬВЕ С ЛЕСНЫМИ ГРИ'],
       [7, 'ЗЕЛПМ-КА32Х62Х17']], dtype=object)

In [62]:
# Test load Data
item_name_train = test_df.loc[test_df.item_name == 'Молоко ДЕРЕВЕНСКОЕ МОЛОЧКО пастер 2.5% п/пак 900мл']
item_name_test = train_df.loc[
            train_df.item_name == 'Молоко ДЕРЕВЕНСКОЕ МОЛОЧКО пастер 2.5% п/пак 900мл']
print(item_name_train, '\n', item_name_test)



          category_id                                          item_name  \
27702571           78  Молоко ДЕРЕВЕНСКОЕ МОЛОЧКО пастер 2.5% п/пак 9...   

          category_id_new  
27702571                0   
        category_id                                          item_name  \
43146           78  Молоко ДЕРЕВЕНСКОЕ МОЛОЧКО пастер 2.5% п/пак 9...   

       category_id_new  
43146              0.0  


Подготовить процессы обработки данных
---------------------------------

Мы пересмотрели самые основные компоненты библиотеки torchtext, включая словарь, векторы слов, токенизатор. Это основные строительные блоки обработки данных для необработанной текстовой строки.

Вот пример типичной обработки данных НЛП с помощью токенизатора и словаря. Первым шагом является создание словаря с помощью необработанного набора обучающих данных. Пользователи могут иметь собственный словарь, задав аргументы в конструкторе класса Vocab. Например, минимальная частота ``min_freq`` для включения токенов.

In [63]:
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab

tokenizer = get_tokenizer('basic_english')
# train_iter = AG_NEWS(split='train')
counter = Counter()
for label, line in train_data:
    counter.update(tokenizer(line))
vocab = Vocab(counter, min_freq=1)

Блок словаря преобразует список токенов в целые числа.

::

    [vocab[token] for token in ['here', 'is', 'an', 'example']]
    >>> [476, 22, 31, 5298]

Подготовьте конвейер обработки текста с токенизатором и словарем. Конвейеры текста и меток будут использоваться для обработки строк необработанных данных из итераторов набора данных.



In [64]:
text_pipeline = lambda x: [vocab[token] for token in tokenizer(x)]
label_pipeline = lambda x: int(x)

Текстовый конвейер преобразует текстовую строку в список целых чисел на основе таблицы поиска, определенной в словаре. Конвейер меток преобразует метку в целые числа. Например,

::

    text_pipeline('here is the an example')
    >>> [475, 21, 2, 30, 5286]
    label_pipeline('10')
    >>> 9




Создать пакет данных и итератор
--------------------------------

`torch.utils.data.DataLoader <https://pytorch.org/docs/stable/data.html?highlight=dataloader#torch.utils.data.DataLoader>`__
рекомендуется для пользователей PyTorch (учебник `here <https://pytorch.org/tutorials/beginner/data_loading_tutorial.html>`__).

Он работает с набором данных в стиле карты, который реализует протоколы ``getitem()`` и ``len()`` и представляет карту от индексов/ключей до образцов данных. Он также работает с повторяющимися наборами данных с аргументом случайного выбора ``False``.


Перед отправкой в модель функция ``collate_fn`` работает с пакетом образцов, сгенерированных из ``DataLoader``. Входом для ``collate_fn`` является пакет данных с размером пакета в ``DataLoader``, и ``collate_fn`` обрабатывает их в соответствии с конвейерами обработки данных, объявленными ранее. Обратите внимание на это и убедитесь, что ``collate_fn`` объявлен как def верхнего уровня. Это гарантирует, что функция доступна каждому исполнителю.

В этом примере текстовые записи в исходном вводе пакета данных упаковываются в список и объединяются в один тензор для ввода ``nn.EmbeddingBag``. Смещение - это тензор разделителей, представляющий начальный индекс отдельной последовательности в текстовом тензоре. Метка - это тензор, сохраняющий метки отдельных текстовых записей.



In [65]:
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)    

dataloader = DataLoader(train_data, batch_size=8, shuffle=False, collate_fn=collate_batch)

Определите модель
----------------

Модель состоит из `nn.EmbeddingBag <https://pytorch.org/docs/stable/nn.html?highlight=embeddingbag#torch.nn.EmbeddingBag>`__ слой плюс линейный слой для целей классификации. ``nn.EmbeddingBag`` с режимом по умолчанию «mean» вычисляет среднее значение «мешка» вложений. Хотя текстовые записи здесь имеют разную длину, модуль nn.EmbeddingBag здесь не требует заполнения, поскольку длина текста сохраняется в смещениях.

Кроме того, поскольку ``nn.EmbeddingBag`` накапливает среднее значение по
вложения на лету, ``nn.EmbeddingBag`` может улучшить
производительность и эффективность памяти для обработки последовательности тензоров.

![](../_static/img/text_sentiment_ngrams_model.png)





In [66]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

Инициировать экземпляр
--------------------

Набор данных ``AG_NEWS`` имеет четыре метки, поэтому количество классов равно четырем.

::

   1 : World мир
   2 : Sports спорт
   3 : Business Бизнес
   4 : Sci/Tec Наука/Техника

Мы строим модель с размером встраивания 64. Размер словаря равен длине экземпляра словаря. Количество классов равно количеству этикеток,




In [67]:
num_class = len(cat_dict)
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [68]:
num_class

98

Определите функции для обучения модели и оценки результатов.
---------------------------------------------------------




In [69]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()
    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
#         print(sorted(label))
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

Разделите набор данных и запустите модель
-----------------------------------

Поскольку исходный ``AG_NEWS`` не имеет действительного набора данных, мы разделяем обучение
набор данных в обучающие/действительные наборы с коэффициентом разделения 0,95 (train) и
0,05 (valid). Здесь мы используем
`torch.utils.data.dataset.random_split <https://pytorch.org/docs/stable/data.html?highlight=random_split#torch.utils.data.random_split>`__
в базовой библиотеке PyTorch.

`CrossEntropyLoss <https://pytorch.org/docs/stable/nn.html?highlight=crossentropyloss#torch.nn.CrossEntropyLoss>`__
критерий объединяет в одном классе ``nn.LogSoftmax()`` и ``nn.NLLLoss()``.
Это полезно при обучении задаче классификации с помощью классов C.
`SGD <https://pytorch.org/docs/stable/_modules/torch/optim/sgd.html>`__
в качестве оптимизатора реализует метод стохастического градиентного спуска. Начальный
скорость обучения установлена на 5.0.
`StepLR <https://pytorch.org/docs/master/_modules/torch/optim/lr_scheduler.html#StepLR>`__
здесь используется для регулировки скорости обучения по эпохам.




In [76]:
from torch.utils.data.dataset import random_split
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # скорость обучения
BATCH_SIZE = 16 # batch size for training
  
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
# train_iter, test_iter = AG_NEWS()
train_dataset = list(train_data)
test_dataset = list(test_data)
num_train = int(len(train_dataset) * 0.9)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

# print(train_dataset)
train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |   500/ 2715 batches | accuracy    0.884
| epoch   1 |  1000/ 2715 batches | accuracy    0.879
| epoch   1 |  1500/ 2715 batches | accuracy    0.880
| epoch   1 |  2000/ 2715 batches | accuracy    0.881
| epoch   1 |  2500/ 2715 batches | accuracy    0.886
-----------------------------------------------------------
| end of epoch   1 | time:  5.96s | valid accuracy    0.876 
-----------------------------------------------------------
| epoch   2 |   500/ 2715 batches | accuracy    0.912
| epoch   2 |  1000/ 2715 batches | accuracy    0.911
| epoch   2 |  1500/ 2715 batches | accuracy    0.908
| epoch   2 |  2000/ 2715 batches | accuracy    0.905
| epoch   2 |  2500/ 2715 batches | accuracy    0.907
-----------------------------------------------------------
| end of epoch   2 | time:  5.80s | valid accuracy    0.874 
-----------------------------------------------------------
| epoch   3 |   500/ 2715 batches | accuracy    0.944
| epoch   3 |  1000/ 2715 batches | accuracy

Запуск модели на GPU со следующей распечаткой:

::

    -----------------------------------------------------------
    | end of epoch   1 | time:  0.03s | valid accuracy    0.450 
    -----------------------------------------------------------
    -----------------------------------------------------------
    | end of epoch   2 | time:  0.03s | valid accuracy    0.450 
    -----------------------------------------------------------
    -----------------------------------------------------------
    | end of epoch   3 | time:  0.03s | valid accuracy    0.450 
    -----------------------------------------------------------
    -----------------------------------------------------------
    | end of epoch   4 | time:  0.03s | valid accuracy    0.650 
    -----------------------------------------------------------
    -----------------------------------------------------------
    | end of epoch   5 | time:  0.02s | valid accuracy    0.700 
    -----------------------------------------------------------
    -----------------------------------------------------------
    | end of epoch   6 | time:  0.02s | valid accuracy    0.700 
    -----------------------------------------------------------
    -----------------------------------------------------------
    | end of epoch   7 | time:  0.02s | valid accuracy    0.700 
    -----------------------------------------------------------
    -----------------------------------------------------------
    | end of epoch   8 | time:  0.02s | valid accuracy    0.700 
    -----------------------------------------------------------
    -----------------------------------------------------------
    | end of epoch   9 | time:  0.02s | valid accuracy    0.700 
    -----------------------------------------------------------
    -----------------------------------------------------------
    | end of epoch  10 | time:  0.02s | valid accuracy    0.750 
    -----------------------------------------------------------



Оцените модель с помощью тестового набора данных
------------------------------------




Проверка результатов тестового набора данных…



In [77]:
print('Проверка результатов тестового набора данных.')
accu_test = evaluate(test_dataloader)
print('точность теста {:8.3f}'.format(accu_test))

Проверка результатов тестового набора данных.
точность теста    0.966


::

    Проверка результатов тестового набора данных.
    точность теста    0.950




Тест на случайной новости
---------------------

Используйте лучшую на данный момент модель и проверьте новости гольфа.




In [78]:
cat_dict_inverted = dict(map(reversed, cat_dict.items()))

def predict(text, text_pipeline = text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item()

ex_text_str = "Сыр ПОШЕХОНСКИЙ 45% вес"

model = model.to("cpu")
predict_cat = predict(ex_text_str, text_pipeline)
print("Это %s категория" %cat_dict_inverted[predict_cat])
exsempl_df = pd.DataFrame(train_data)
print('Примеры:\n', exsempl_df[exsempl_df[0] == predict_cat][1][:50].values)

Это 78 категория
Примеры:
 ['Молоко 3,2%,шт' 'Молоко пастерилиз. т/рекс 2,5 %  1 л. 1/8 БМК'
 'Сметана  Кубанский  Молочник  20%  180гр шт' 'СМЕТАНА 20% 300Г'
 'Молоко Мудрая хозяйка 2,5% пленка 0,8 л БЗМЖ'
 '45Г ГЛ/СЫРКИ РОСТАГРО 20% ВН БЗМЖ' 'Пакет'
 'Биойогурт "Козимель" натуральный,шт'
 'Продукт кисломолочный Тема Биолакт 3,2% ТБА 208г'
 'Йогурт питьевой Асеньевская фе' 'Сыр Российский 1кг 50% Россия БЗМЖ'
 'Сметана 10г, 16 ккал' ' К.Ц.Яйца куриные стол.С2 10шт' 'ЯЙЦО ЭКСТРА С0'
 'ЯЙЦО КУРИНОЕ С1 ВАРА' 'Йогурт фруктовый "Вишня" 2,5%,шт'
 'Молоко Отборн Грин Агро 3,3-4,3% 1л пэт ГМЗ Артем'
 'Молоко ФП пастериз т/топ 2,5 % 1,0л 1/6 Хладокомбинат'
 'МАСЛО 82,5% ЭКОМ.' 'Яйцо кур С2 10шт пл/бл Племптицезавод'
 'Мильфей с клубн.тартаром'
 'БЗМЖ Сыр Сметанковый 45 - 50% 250г кусок Продукты из Елани'
 'Сметана 20%,шт' 'ЯЙЦО СТОЛОВОЕ 1КАТ. ПТИЦЕФАБРИКА ПИОНЕРСКОЕ 10ШТ'
 'Сыр с аром топл мол Черный принц 50% Беларусь'
 'Коктейль детский молочный c шоколадом 2,5%,шт' 'МОЛОКО ПМЖЛ 2.5% 900Г'
 

In [79]:
# df_test = pd.DataFrame({'item_name' : ['Лук', 'Хрен', 'Суп семи залуп', 'АИ95']})
df_test = pd.read_pickle('../data/input/test_data_6350_rows.pkl')[['category_id', 'item_name']]
df_test['Pred'] = df_test.item_name.apply(predict).map(dict(map(reversed, cat_dict.items())))
df_test[df_test.category_id != df_test.Pred]

,category_id,item_name,Pred
32644219,84,Пакет,203
27416567,203,ЗЕЛПМ-КА32Х62Х17,138
9398547,71,Макароны 200 гр.,76
31102586,71,"Компот 0,2л.",83
33164615,74,"Окорочка куриные гриль, вес",79
...,...,...,...
16527974,78,Пакет,203
43633645,73,Печень говяжья п/ф,79
35226044,77,"Майонез Домашний 67% 420мл Лу-ка Россия, шт",78
15224849,203,ЗЕЛПМ-КА32Х62Х17,138


::

       This is a Sports news


